In [1]:
import pandas as pd
import jieba

data = pd.read_excel('带标签的论文摘要.xlsx')

# 加载停用词表
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = set(file.read().strip().split('\n'))
    return stopwords

stopwords = load_stopwords('stopwords.txt')

# 分词并去除停用词
def jieba_tokenizer(text):
    words = jieba.cut(text)
    return ' '.join([word for word in words if word not in stopwords])

data['tokenized_abstract'] = data['论文摘要'].apply(jieba_tokenizer)

print(data['tokenized_abstract'].head())

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\60938\AppData\Local\Temp\jieba.cache
Loading model cost 0.706 seconds.
Prefix dict has been built successfully.


0    目标 检测 计算机 视觉 中 一个 重要 问题 , 行人 跟踪 车牌 识别 无人驾驶 领域 ...
1    目标 检测 计算机 视觉 一个 重要 研究 方向 , 目的 精确 识别 给定 图像 中 特定...
2    目标 检测 计算机 视觉 中 核心 任务 , 智能 视频 监控 自动化 监测 工业 检测 领...
3    路径 规划 移动机器人 热门 研究 , 实现 机器人 自主 导航 关键技术 移动机器人 路径...
4    目标 检测 计算机 视觉 核心 , 图像识别 行人 检测 大规模 场景 识别 方面 具有 广...
Name: tokenized_abstract, dtype: object


In [2]:
# 使用TF-IDF向量化
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['tokenized_abstract'])
y = data['标签']

print(X.shape, y.shape)

(200, 2191) (200,)


In [3]:
from sklearn.feature_selection import SelectKBest, chi2

k = 1000
selector = SelectKBest(chi2, k=k)
X_kbest = selector.fit_transform(X, y)

print(X_kbest.shape)

(200, 1000)


In [6]:
from sklearn.decomposition import TruncatedSVD

n_components = 100
svd = TruncatedSVD(n_components=n_components)
X_reduced = svd.fit_transform(X_kbest)

print(X_reduced.shape)

(200, 100)


In [7]:
# 保存处理后的数据
import numpy as np

np.save('特征储存.npy', X_reduced)
np.save('标签储存.npy', y.values)